# Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


## Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Object tracking function

In [8]:
def set_baseline(x1, y1, x2, y2):
    return y2 - y1, x1 - x2, x2 * y1 - x1 * y2

In [45]:
def get_distance_from_line(x1, y1, a, b, c):
    return (a*x1 + b*y1 + c) / (((a ** 2) + (b ** 2)) ** 0.5)

# Baseline setting function

In [61]:
baseline_x = []
baseline_y = []
mouse_count = 2

def get_mouse_click_location(event, x, y, flags, param):
    global mouse_count, a, b, c
    if (mouse_count > 0 and event == cv2.EVENT_LBUTTONDOWN):
        baseline_x.append(x)
        baseline_y.append(y)
        mouse_count = mouse_count - 1
        if (mouse_count == 0):
            a, b, c = set_baseline(baseline_x[0] * 0.001, baseline_y[0] * 0.001, baseline_x[1] * 0.001, baseline_y[1] * 0.001)

# Waring signal function

In [62]:
standard_distance = 0.03

def send_warning_signal(input_distance, input_class):
    if standard_distance > input_distance:
        if input_class == 'person':
            return 0
        else:
            return 1
    return -1

# STORED VIDEO ONLY

In [64]:
inputVideo = '.\\video_samples\\s1_a.mp4'
cap=cv2.VideoCapture(inputVideo)
with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        if len(baseline_x) == 2:
            cv2.imshow('stored version', cv2.line(image_np,(baseline_x[0], baseline_y[0]),(baseline_x[1], baseline_y[1]),(0,0,255),5))
            
            for index, value in enumerate(classes[0]):
                ymin = boxes[0][index][0]
                xmin = boxes[0][index][1]
                ymax = boxes[0][index][2]
                xmax = boxes[0][index][3]
                class_name = (category_index.get(value)).get('name')
                widthvalue = int((xmax - xmin) / 2)  # width 길이
                heightvalue = int((ymax - ymin) / 2)  # height 길이
                if ((class_name == "person" or class_name == "car") and scores[0, index] > 0.80) :
                    #print((xmin + xmax) / 2, (ymin + ymax) / 2) #중심 좌표
                    print(get_distance_from_line((xmin + xmax) / 2, (ymin + ymax) / 2, a, b, c)) # 선으로부터 점 사이 거리
                    print(class_name + " : " + str(index)) # 클래스의 이름과 인덱스
                    
                    # 기준 거리 내 사람일 경우 0, 차일 경우 1
                    print(send_warning_signal(get_distance_from_line((xmin + xmax) / 2, (ymin + ymax) / 2, a, b, c), class_name))
                    print("\n")

        cv2.imshow('stored version', image_np)
        
        # Mouse Click 좌표를 위한 함수
        cv2.setMouseCallback('stored version', get_mouse_click_location)
        
        if cv2.waitKey(25) & 0xFF==ord('q'): #waitKey( 내부의 값이 작아지면 CPU 의 부담은 커지는데 비해 처리속도는 빨라짐 )
            break

        # 저장된 비디오의 경우 한정
        if(cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT)):
            break
        
    cv2.destroyAllWindows()
    cap.release()

0.15877154180749198
person : 0
-1


0.15708043666235216
person : 0
-1


0.15724375305406418
person : 0
-1


0.157129816572488
person : 0
-1


0.1560747589992146
person : 0
-1


0.11127278288099869
person : 0
-1


0.10677154353135053
person : 0
-1


0.0996245191576786
person : 0
-1


0.09971301574053988
person : 0
-1


0.0967605296250204
person : 0
-1


0.056178240737257024
person : 0
-1


0.05629334092449235
person : 0
-1


0.04938669288477703
person : 0
-1


0.046673765276737014
person : 0
-1


0.04266789926360279
person : 0
-1


0.039964995686809904
person : 0
-1


0.039930501511563854
person : 0
-1


0.003973978176522853
person : 0
0


0.0013758695268505784
person : 0
0


-0.0028887173469501603
person : 0
0


-0.002858170073397959
person : 0
0


-0.037065142762827
person : 0
0


-0.036848282980732054
person : 0
0


-0.04315745867492332
person : 0
0


-0.051933945735216126
person : 0
0


-0.056977998822082106
person : 0
0


-0.0719453967564412
person : 0
0


-0.09185216598859586
pers

# STREAMING ONLY

In [ ]:
cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach
with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        for index, value in enumerate(classes[0]):
            ymin = boxes[0][index][0]
            xmin = boxes[0][index][1]
            ymax = boxes[0][index][2]
            xmax = boxes[0][index][3]
            class_name = (category_index.get(value)).get('name')
            widthvalue = int((xmax - xmin) / 2)  # width 길이
            heightvalue = int((ymax - ymin) / 2)  # height 길이
            if ((class_name == "person" or class_name == "car") and scores[0, index] > 0.80) :
                print((xmin + xmax) / 2, (ymin + ymax) / 2)
                print(class_name  + str(index) + "\n")
                
        cv2.imshow('streaming version', image_np)
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
        
    cv2.destroyAllWindows()
    cap.release()

In [35]:
import numpy as np   # for zeros
import cv2 as cv


# 1. 마우스 이벤트 발생시 호출될 함수를 정의합니다. 
def mouse_callback(event, x, y, flags, param):
    print("마우스 이벤트 발생")


img = np.zeros((512, 512, 3), np.uint8)
cv.namedWindow('image')  # 2. 마우스 이벤트를 감지할 윈도우를 생성합니다.  

# 3. 이름이 image인 윈도우에서 마우스 이벤트가 발생하면 mouse_callback 함수가 호출되게 됩니다. 
cv.setMouseCallback('image', mouse_callback)  


cv.imshow('image',img)
cv.waitKey(0)

cv.destroyAllWindows()

마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생
마우스 이벤트 발생